1. **Rank the customers based on the total amount they've spent on rentals.**

Objective: Calculate the difference in rental counts between each customer's total rentals and the average rentals across all customers
- Concept : Calculate the customer's total rentals ($\text{COUNT()}$ per customer) and then use the $\text{AVG()}$ window function without a $\text{PARTITION BY}$ clause to get the overall average rental count across all customers. Subtract the average from the individual count.

In [ ]:
WITH CustomerRentals AS (
    SELECT
        customer_id,
        COUNT(rental_id) AS customer_rental_count
    FROM
        rental
    GROUP BY
        customer_id
)
SELECT
    customer_id,
    customer_rental_count,
    AVG(customer_rental_count) OVER () AS avg_rentals_all_customers,
    customer_rental_count - AVG(customer_rental_count) OVER () AS difference_from_average
FROM
    CustomerRentals;

2. **Calculate the cumulative revenue generated by each film over time.**


Objective: Calculate the cumulative revenue generated by each film over time.

- Concept: Use the $\text{SUM()}$ window function, partitioned by $\text{film\_id}$ and ordered by $\text{payment\_date}$, to create a running total of revenue for each film.

In [ ]:
SELECT
    p.payment_date,
    i.film_id,
    p.amount,
    SUM(p.amount) OVER (
        PARTITION BY i.film_id
        ORDER BY p.payment_date
    ) AS cumulative_revenue
FROM
    payment p
JOIN
    rental r ON p.rental_id = r.rental_id
JOIN
    inventory i ON r.inventory_id = i.inventory_id
ORDER BY
    i.film_id, p.payment_date;

3. **Determine the average rental duration for each film, considering films with similar lengths.**


Objective: Determine the average rental duration for each film, considering films with similar lengths.

- Concept: Use a $\text{CASE}$ statement or the $\text{NTILE()}$ function to group films into "buckets" (e.g., Short, Medium, Long) based on their $\text{length}$. Then, calculate the $\text{AVG()}$ rental duration for each bucket.

- Key SQL (using $\text{CASE}$ for length buckets)


In [ ]:
WITH FilmDuration AS (
    SELECT
        r.rental_duration,
        f.length,
        CASE
            WHEN f.length <= 60 THEN 'Short'
            WHEN f.length > 60 AND f.length <= 120 THEN 'Medium'
            ELSE 'Long'
        END AS length_group
    FROM
        rental r
    JOIN
        inventory i ON r.inventory_id = i.inventory_id
    JOIN
        film f ON i.film_id = f.film_id
)
SELECT
    length_group,
    AVG(rental_duration) AS average_rental_duration
FROM
    FilmDuration
GROUP BY
    length_group;

4. **Identify the top 3 films in each category based on their rental counts.**


Objective: Identify the top 3 films in each category based on their rental counts.

- Concept: This is a "Top N per Group" problem. Calculate the rental count per film, and then use the $\text{ROW\_NUMBER()}$ or $\text{RANK()}$ window function $\text{PARTITIONED BY category}$ to rank films within their category. Filter for ranks 1, 2, and 3.

In [ ]:
WITH RankedFilms AS (
    SELECT
        f.title,
        c.name AS category_name,
        COUNT(r.rental_id) AS rental_count,
        ROW_NUMBER() OVER (
            PARTITION BY c.name
            ORDER BY COUNT(r.rental_id) DESC
        ) AS category_rank
    FROM
        rental r
    JOIN
        inventory i ON r.inventory_id = i.inventory_id
    JOIN
        film f ON i.film_id = f.film_id
    JOIN
        film_category fc ON f.film_id = fc.film_id
    JOIN
        category c ON fc.category_id = c.category_id
    GROUP BY
        f.title, c.name
)
SELECT
    category_name,
    title,
    rental_count
FROM
    RankedFilms
WHERE
    category_rank <= 3
ORDER BY
    category_name, rental_count DESC;

5. **Calculate the difference in rental counts between each customer's total rentals and the average rentals
across all customers.**


Objective: Calculate the difference in rental counts between each customer's total rentals and the average rentals across all customers.

- Concept: Calculate the customer's total rentals ($\text{COUNT()}$ per customer) and then use the $\text{AVG()}$ window function without a $\text{PARTITION BY}$ clause to get the overall average rental count across all customers. Subtract the average from the individual count.

In [ ]:
WITH CustomerRentals AS (
    SELECT
        customer_id,
        COUNT(rental_id) AS customer_rental_count
    FROM
        rental
    GROUP BY
        customer_id
)
SELECT
    customer_id,
    customer_rental_count,
    AVG(customer_rental_count) OVER () AS avg_rentals_all_customers,
    customer_rental_count - AVG(customer_rental_count) OVER () AS difference_from_average
FROM
    CustomerRentals;

6. **Find the monthly revenue trend for the entire rental store over time.**


Objective: Find the monthly revenue trend for the entire rental store over time.

- Concept: Group the payments by year and month (using a function like $\text{DATE\_TRUNC}$ or $\text{EXTRACT}$), and then use $\text{SUM()}$ to aggregate the total revenue for that period.

In [ ]:
SELECT
    DATE_TRUNC('month', payment_date) AS rental_month,
    SUM(amount) AS monthly_revenue
FROM
    payment
GROUP BY
    rental_month
ORDER BY
    rental_month;

7. **Identify the customers whose total spending on rentals falls within the top 20% of all customers.**


Objective: Identify the customers whose total spending on rentals falls within the top 20% of all customers.

- Concept: Calculate total spending per customer. Use the $\text{NTILE(5)}$ window function to divide the customers into 5 equal groups (quintiles). Customers in the first tile ($\text{NTILE} = 1$) represent the top 20%.

In [ ]:
WITH CustomerSpendingTile AS (
    SELECT
        customer_id,
        SUM(amount) AS total_spent,
        NTILE(5) OVER (ORDER BY SUM(amount) DESC) AS spending_quintile
    FROM
        payment
    GROUP BY
        customer_id
)
SELECT
    customer_id,
    total_spent
FROM
    CustomerSpendingTile
WHERE
    spending_quintile = 1
ORDER BY
    total_spent DESC;

8. **Calculate the running total of rentals per category, ordered by rental count.**


Objective: Calculate the running total of rentals per category, ordered by rental count.

- Concept: Calculate the total rental count for each category. Then, apply a $\text{SUM()}$ window function over these counts, ordered by the count itself, to get a running total across the categories.

In [ ]:
WITH CategoryRentals AS (
    SELECT
        c.name AS category_name,
        COUNT(r.rental_id) AS rental_count
    FROM
        rental r
    JOIN
        inventory i ON r.inventory_id = i.inventory_id
    JOIN
        film_category fc ON i.film_id = fc.film_id
    JOIN
        category c ON fc.category_id = c.category_id
    GROUP BY
        c.name
)
SELECT
    category_name,
    rental_count,
    SUM(rental_count) OVER (ORDER BY rental_count DESC) AS running_total_rentals
FROM
    CategoryRentals
ORDER BY
    rental_count DESC;

9. **Find the films that have been rented less than the average rental count for their respective categories.**


Objective: Find the films that have been rented less than the average rental count for their respective categories.

- Concept: Calculate the individual film rental count and the category-wide average rental count using $\text{AVG()}$ partitioned by category. Then, filter for films where the individual count is less than the category average.

In [ ]:
WITH FilmCategoryRentals AS (
    SELECT
        f.film_id,
        f.title,
        c.name AS category_name,
        COUNT(r.rental_id) AS film_rental_count
    FROM
        rental r
    JOIN
        inventory i ON r.inventory_id = i.inventory_id
    JOIN
        film f ON i.film_id = f.film_id
    JOIN
        film_category fc ON f.film_id = fc.film_id
    JOIN
        category c ON fc.category_id = c.category_id
    GROUP BY
        f.film_id, f.title, c.name
)
SELECT
    title,
    category_name,
    film_rental_count,
    AVG(film_rental_count) OVER (PARTITION BY category_name) AS avg_category_rentals
FROM
    FilmCategoryRentals
WHERE
    film_rental_count < AVG(film_rental_count) OVER (PARTITION BY category_name)
ORDER BY
    category_name, film_rental_count;

10. **Identify the top 5 months with the highest revenue and display the revenue generated in each month.**

Objective: Identify the top 5 months with the highest revenue and display the revenue generated in each month.

- Concept: Aggregate revenue by month (similar to Q6). Then, $\text{ORDER BY}$ revenue in descending order and use $\text{LIMIT 5}$ to select the top months.

In [ ]:
SELECT
    DATE_TRUNC('month', payment_date) AS rental_month,
    SUM(amount) AS monthly_revenue
FROM
    payment
GROUP BY
    rental_month
ORDER BY
    monthly_revenue DESC
LIMIT 5;